In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re

from scipy.fftpack import fft, rfft, fftfreq
from scipy.io.wavfile import write
from scipy import log10
from pylab import *

%matplotlib inline

plt.rcParams["figure.figsize"] = (12,9)

In [3]:
# load pickle file for dataframe
df = pickle.load(open( "lyincomey.p", "rb" ))

In [5]:
df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user,withheld_in_countries
0,None,None,Tue Apr 17 14:59:37 +0000 2018,"{'hashtags': [{'text': 'ComeyBaloney', 'indice...",NaN,0,False,None,986257883351408640,986257883351408640,...,NaN,NaN,33,False,{'created_at': 'Tue Apr 17 14:21:59 +0000 2018...,"a href=""http://twitter.com/#!/download/ipad"" r...",RT @LadyRedStorm: Ha SO True! \n\n#ComeyBalone...,False,"{'id': 513876919, 'id_str': '513876919', 'name...",NaN
1,None,None,Tue Apr 17 14:59:36 +0000 2018,"{'hashtags': [{'text': 'ComeyBaloney', 'indice...",NaN,0,False,None,986257876757991425,986257876757991425,...,NaN,NaN,33,False,{'created_at': 'Tue Apr 17 14:21:59 +0000 2018...,"a href=""http://twitter.com"" rel=""nofollow""Twit...",RT @LadyRedStorm: Ha SO True! \n\n#ComeyBalone...,False,"{'id': 1241009113, 'id_str': '1241009113', 'na...",NaN
2,None,None,Tue Apr 17 14:58:18 +0000 2018,"{'hashtags': [{'text': 'ComeyBaloney', 'indice...",NaN,0,False,None,986257551745519618,986257551745519618,...,NaN,NaN,33,False,{'created_at': 'Tue Apr 17 14:21:59 +0000 2018...,"a href=""http://twitter.com"" rel=""nofollow""Twit...",RT @LadyRedStorm: Ha SO True! \n\n#ComeyBalone...,False,"{'id': 2823913560, 'id_str': '2823913560', 'na...",NaN
3,None,None,Tue Apr 17 14:56:06 +0000 2018,"{'hashtags': [{'text': 'FBI', 'indices': [36, ...",NaN,0,False,None,986256997782818816,986256997782818816,...,NaN,NaN,8,False,{'created_at': 'Tue Apr 17 14:49:41 +0000 2018...,"a href=""http://twitter.com/download/iphone"" re...",RT @CuteScarley: Disgraced Partisan #FBI Agent...,False,"{'id': 3291902538, 'id_str': '3291902538', 'na...",NaN
4,None,None,Tue Apr 17 14:55:35 +0000 2018,"{'hashtags': [{'text': 'ComeyBaloney', 'indice...",NaN,0,False,None,986256867516211201,986256867516211201,...,NaN,NaN,33,False,{'created_at': 'Tue Apr 17 14:21:59 +0000 2018...,"a href=""http://twitter.com/download/iphone"" re...",RT @LadyRedStorm: Ha SO True! \n\n#ComeyBalone...,False,"{'id': 860776572, 'id_str': '860776572', 'name...",NaN


In [6]:
allcolumns = df.columns

for col in allcolumns:
    print(col)

contributors
coordinates
created_at
entities
extended_entities
favorite_count
favorited
geo
id
id_str
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
is_quote_status
lang
metadata
place
possibly_sensitive
quoted_status
quoted_status_id
quoted_status_id_str
retweet_count
retweeted
retweeted_status
source
text
truncated
user
withheld_in_countries


In [22]:
columns_subset = ['created_at', 'favorite_count', 'geo', 'lang', 'place', 'id', 'retweet_count', 'retweeted', 'source', 'text']

In [40]:
# create new dataframe based on data subset
df2 = df.loc[:,columns_subset]
df2.head()

# reindex selection by created at timestamp
df2.set_index('created_at', inplace=True)
df2.sort_index(inplace=True)

# Converting the index as date
df2.index = pd.to_datetime(df2.index)

df2.head()

,favorite_count,geo,lang,place,id,retweet_count,retweeted,source,text
created_at,,,,,,,,,
2018-04-13 00:07:51,0,None,und,None,984583909294923779,63,False,"a href=""http://twitter.com/download/android"" r...",RT @B75434425: #LyinComey https://t.co/fHlc6u56Q3
2018-04-13 00:07:52,0,None,en,None,984583914009350144,61,False,"a href=""http://www.twitter.com"" rel=""nofollow""...",RT @rcjhawk86: #LyinComey #FBICorruption\n\nIt...
2018-04-13 00:07:52,0,None,en,None,984583914286088192,2,False,"a href=""http://twitter.com"" rel=""nofollow""Twit...",RT @littlebitsnbob4: @seanhannity @realDonaldT...
2018-04-13 00:08:00,0,None,en,None,984583949014925317,1,False,"a href=""http://twitter.com"" rel=""nofollow""Twit...","RT @watchingfoxes: Comey calls Loretta Lynch, ..."
2018-04-13 00:08:11,0,None,und,None,984583994003075072,3,False,"a href=""https://mobile.twitter.com"" rel=""nofol...",RT @peidays306: #LyinComey #MAGA https://t.co...


In [24]:
len(df2)

15844

In [25]:
# attempt to pull out any duplicates based on text data
rt_set = df2['text'].value_counts().index.tolist()
rt_set_vals = df2['text'].value_counts().values.tolist()


In [26]:
len(rt_set)

2170

In [32]:
# this means there are only 2170 unique tweet text our of the 15844?

In [33]:
rt_set2 = df2['id'].value_counts().index.tolist()

In [48]:
len(rt_set2)

9078

In [56]:
first_time = df2.index[0]

In [57]:
first_time

Timestamp('2018-04-13 00:07:51')

In [62]:
df3 = df2

# set initial timestamp as first entry
initial_time = df3.index[0]

df3['seconds'] = (df3.index - initial_time).total_seconds()
df3.head()

,favorite_count,geo,lang,place,id,retweet_count,retweeted,source,text,seconds
created_at,,,,,,,,,,
2018-04-13 00:07:51,0,None,und,None,984583909294923779,63,False,"a href=""http://twitter.com/download/android"" r...",RT @B75434425: #LyinComey https://t.co/fHlc6u56Q3,0.0
2018-04-13 00:07:52,0,None,en,None,984583914009350144,61,False,"a href=""http://www.twitter.com"" rel=""nofollow""...",RT @rcjhawk86: #LyinComey #FBICorruption\n\nIt...,1.0
2018-04-13 00:07:52,0,None,en,None,984583914286088192,2,False,"a href=""http://twitter.com"" rel=""nofollow""Twit...",RT @littlebitsnbob4: @seanhannity @realDonaldT...,1.0
2018-04-13 00:08:00,0,None,en,None,984583949014925317,1,False,"a href=""http://twitter.com"" rel=""nofollow""Twit...","RT @watchingfoxes: Comey calls Loretta Lynch, ...",9.0
2018-04-13 00:08:11,0,None,und,None,984583994003075072,3,False,"a href=""https://mobile.twitter.com"" rel=""nofol...",RT @peidays306: #LyinComey #MAGA https://t.co...,20.0


In [63]:
# write out csv file of all data
df3.to_csv('lyincomey_subset_v5_wseconds.csv')

In [67]:
jsondata = df3.to_json(orient='records')

# write out new master json file
with open("_df_subset_v5_wseconds.json", "w") as outfile:
    json.dump(jsondata, outfile)